In [ ]:
!pip install Whoosh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import StandardAnalyzer

from whoosh import qparser
from whoosh import scoring
import whoosh.index as index

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Reading Corpus

In [ ]:
# NFCorpus
!gdown 1QscBtuFj3XW7Gl3lhWdEx6r8Yit0yJ2v

Downloading...
From: https://drive.google.com/uc?id=1QscBtuFj3XW7Gl3lhWdEx6r8Yit0yJ2v
To: /content/nfcorpus.tar.gz
100% 97.3M/97.3M [00:00<00:00, 167MB/s]


In [ ]:
!tar -xvf 'nfcorpus.tar.gz'

nfcorpus/
nfcorpus/train.docs
nfcorpus/test.docs
nfcorpus/dev.docs
nfcorpus/dev.3-2-1.qrel
nfcorpus/test.3-2-1.qrel
nfcorpus/train.3-2-1.qrel
nfcorpus/raw/
nfcorpus/raw/doc_dump.txt
nfcorpus/raw/dev.docs.ids
nfcorpus/raw/dev.queries.ids
nfcorpus/raw/test.docs.ids
nfcorpus/raw/test.queries.ids
nfcorpus/raw/train.docs.ids
nfcorpus/raw/train.queries.ids
nfcorpus/raw/stopwords.large
nfcorpus/raw/nfdump.txt
nfcorpus/raw/all_videos.ids
nfcorpus/raw/nontopics.ids
nfcorpus/test.2-1-0.qrel
nfcorpus/dev.2-1-0.qrel
nfcorpus/train.2-1-0.qrel
nfcorpus/ecir2016.bib
nfcorpus/test.vid-desc.queries
nfcorpus/dev.vid-desc.queries
nfcorpus/train.vid-desc.queries
nfcorpus/dev.all.queries
nfcorpus/test.all.queries
nfcorpus/train.all.queries
nfcorpus/dev.vid-titles.queries
nfcorpus/test.vid-titles.queries
nfcorpus/train.vid-titles.queries
nfcorpus/dev.titles.queries
nfcorpus/test.titles.queries
nfcorpus/train.titles.queries
nfcorpus/dev.nontopic-titles.queries
nfcorpus/test.nontopic-titles.queries
nfcorpus/t

In [ ]:
## Read the .docs file
CorpusDocs = {}

with open('/content/nfcorpus/train.docs', 'r', encoding='utf-8') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()
        doc_id, doc_text = line.split('\t')
        CorpusDocs[doc_id] = doc_text

# Now you have the documents stored in the 'documents' dictionary

In [ ]:
# Read any queries
CorpusQueries = {}

with open('/content/nfcorpus/train.all.queries', 'r', encoding='utf-8') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()
        query_id, query_text = line.split('\t')
        CorpusQueries[query_id] = query_text

# Now you have the queries stored in the 'queries' dictionary
CorpusQueries = dict(sorted(CorpusQueries.items()))


In [ ]:
# Read relevance judgements

CorpusRelJudgements = {}

with open('/content/nfcorpus/train.3-2-1.qrel', 'r', encoding='utf-8') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()
        query_id, _, doc_id, relevance = line.split('\t')

        if query_id not in CorpusRelJudgements:
            CorpusRelJudgements[query_id] = {}

        CorpusRelJudgements[query_id][doc_id] = int(relevance)

# Now you have the relevance judgments stored in the 'relevance_judgments' dictionary
corpus_result_que = {}
for i in CorpusRelJudgements:
  res = []
  for j in CorpusRelJudgements[i]:
    res.append(j)
  corpus_result_que[i] = res

corpus_result_que = dict(sorted(corpus_result_que.items()))

# Functions

In [ ]:
# Trang ver of AP/mAP by returning just the main mAP value of all 225 queries

def precision(y_true, y_pred):
  prec_list = []
  rec_list = []
  count = 0
  for i in range(len(y_pred)):
    if y_pred[i] in y_true:
      count += 1
      prec_list.append(count/(i+1))
      rec_list.append(count/len(y_true))
  return prec_list, rec_list
# list prec nội suy nè
def prec_in(prec_list, rec_list):
  prec_list_in = []
  for i in range(11):
    rec = i/10
    # prec_rec = max(rec->all)
    # lấy max từ prec_rec_in
    prec_rec_in = next((x[0] for x in enumerate(rec_list) if x[1] >= rec),-1)
    # print('rec_in:', prec_rec_in)
    prec_rec = 0
    if prec_rec_in != -1:
      prec_rec = max(prec_list[prec_rec_in:])
    prec_list_in.append(prec_rec)
  return prec_list_in

In [ ]:
def MAP(list_y_true, list_y_pred, noisuy=True):
    common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
    result_dict = {}

    if noisuy:
        prec_in_data = {}
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            prec_in_list = prec_in(prec_list, rec_list)
            # print('prec_list', prec_list)
            # print('rec_list', rec_list)
            # print('prec_list_in', prec_in_list)
            result_dict[key] = sum(prec_in_list) / len(prec_in_list)
            prec_in_data[key] = prec_in_list
        return prec_in_data, result_dict, sum(result_dict.values()) / len(result_dict.values())
    else:
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            if len(prec_list) != 0:
                result_dict[key] = sum(prec_list) / len(prec_list)
            else:
                result_dict[key] = 0

    return result_dict, sum(result_dict.values()) / len(result_dict.values())

In [ ]:
def count_P_R(list_y_true, list_y_pred):
  common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
  result_P_dict = {}
  result_R_dict = {}
  for key in common_keys:
      count = 0
      for i in range(len(list_y_pred[key])):
        if list_y_pred[key][i] in list_y_true[key]:
          count += 1
      if len(list_y_pred[key])!=0:
        result_P_dict[key] = count / len(list_y_pred[key])
      else:
        result_P_dict[key] = 0
      result_R_dict[key] = count / len(list_y_true[key])
  return result_P_dict, result_R_dict, sum(result_P_dict.values()) / len(result_P_dict.values()), sum(result_R_dict.values()) / len(result_R_dict.values())

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
import string

def preprocessQueryStemmingAnalyzer(query_text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Lowercase the query text
    query_text = query_text.lower()

    # Remove non-alphanumeric characters
    query_text = re.sub(r'[^a-z0-9\s]', '', query_text)

    # Tokenize the query text
    words = word_tokenize(query_text)

    # Process each word in the query
    processed_words = []
    for word in words:
        if word not in stop_words and word.isdigit() == False and word.isnumeric() == False and word.isalnum():
            stemmed_word = stemmer.stem(word)
            processed_words.append(stemmed_word)

    # Join the processed words into a single string
    processed_query = ' '.join(processed_words)

    return processed_query


In [ ]:
def preprocessQueryWithStopwords(query_text):
    stemmer = PorterStemmer()

    # Lowercase the query text
    query_text = query_text.lower()

    # Remove non-alphanumeric characters
    query_text = re.sub(r'[^a-z0-9\s]', '', query_text)

    # Tokenize the query text
    words = word_tokenize(query_text)

    # Process each word in the query
    processed_words = []
    for word in words:
        stemmed_word = stemmer.stem(word)
        processed_words.append(stemmed_word)

    # Join the processed words into a single string
    processed_query = ' '.join(processed_words)

    return processed_query

# Stem, Remove Stop

In [ ]:
# import shutil

# shutil.rmtree('/content/indCorpusPSRemoveStop')

In [ ]:
!mkdir /content/indCorpusPSRemoveStop

In [ ]:
from whoosh.analysis import StandardAnalyzer, StopFilter
from whoosh.analysis import StemmingAnalyzer

import time

start_time = time.time()

schema = Schema(docname=ID(stored=True), content=TEXT(stored=True, analyzer=StemmingAnalyzer()))
ix = create_in("indCorpusPSRemoveStop", schema)
writer = ix.writer()

for docname, content in CorpusDocs.items():
    writer.add_document(docname=docname, content=content)

writer.commit()

total_timeStemRemoveStop = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop)


Total execution time: 15.518402099609375


In [ ]:
!ls /content/indCorpusPSRemoveStop

_MAIN_1.toc  MAIN_oyb8fjjparwc0cxe.seg	MAIN_WRITELOCK


## TF-IDF

In [ ]:
ind = index.open_dir("indCorpusPSRemoveStop")
searcher = ind.searcher(weighting=scoring.TF_IDF())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}

start_time = time.time()

for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)
    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_TFIDF = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_TFIDF)

Total execution time: 1087.5343987941742


In [ ]:
len(results_dict)

2594

In [ ]:
listAP_PSRemoveStop_TFIDF, listOfAP_PSRemoveStop_TFIDF_Inter, MAP_PSRemoveStop_TFIDF_Inter = MAP(corpus_result_que, results_dict)
l_PSRemoveStop_TFIDF_p5, l_PSRemoveStop_TFIDF_r5, p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF = count_P_R(corpus_result_que, results_dict)
listOfAP_PSRemoveStop_TFIDF_NonInter, MAP_PSRemoveStop_TFIDF_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PSRemoveStop_TFIDF_Inter)
print(MAP_PSRemoveStop_TFIDF_NonInter)

0.10888477796352161
0.10357016026745498


In [ ]:
print(p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF)

0.018414110102097207 0.8913896918991252


## BM25F

In [ ]:
ind = index.open_dir("indCorpusPSRemoveStop")
searcher = ind.searcher(weighting=scoring.BM25F())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}

start_time = time.time()

for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)
    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_BM25F = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_BM25F)

Total execution time: 1425.457442998886


In [ ]:
listAP_PSRemoveStop_BM25F, listOfAP_PSRemoveStop_BM25F_Inter, MAP_PSRemoveStop_BM25F_Inter = MAP(corpus_result_que, results_dict)
l_PSRemoveStop_BM25F_p5, l_PSRemoveStop_BM25F_r5, p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F = count_P_R(corpus_result_que, results_dict)
listOfAP_PSRemoveStop_BM25F_NonInter, MAP_PSRemoveStop_BM25F_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PSRemoveStop_BM25F_Inter)
print(MAP_PSRemoveStop_BM25F_NonInter)


0.17486483217037657
0.16731848635046218


In [ ]:
print(p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F)

0.01843016042640293 0.8927512012672214


## Frequency

In [ ]:
ind = index.open_dir("indCorpusPSRemoveStop")
searcher = ind.searcher(weighting=scoring.Frequency())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()
for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)
    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_Frequency = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_Frequency)

Total execution time: 880.9008748531342


In [ ]:
listAP_PSRemoveStop_Frequency, listOfAP_PSRemoveStop_Frequency_Inter, MAP_PSRemoveStop_Frequency_Inter = MAP(corpus_result_que, results_dict)
l_PSRemoveStop_Frequency_p5, l_PSRemoveStop_Frequency_r5, p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency = count_P_R(corpus_result_que, results_dict)
listOfAP_PSRemoveStop_Frequency_NonInter, MAP_PSRemoveStop_Frequency_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PSRemoveStop_Frequency_Inter)
print(MAP_PSRemoveStop_Frequency_NonInter)

0.06702140610963818
0.06455814247671546


In [ ]:
print(p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency)

0.0182784254160683 0.8817167781328643


# Stem Only

In [ ]:
# shutil.rmtree('/content/indCorpusPS')

In [ ]:
!mkdir /content/indCorpusPS

In [ ]:
start_time = time.time()

schema = Schema(docname=ID(stored=True), content=TEXT(stored=True, analyzer=StemmingAnalyzer(stoplist=None)))
ix = create_in("indCorpusPS", schema)
writer = ix.writer()

for docname, content in CorpusDocs.items():
    writer.add_document(docname=docname, content=content)

writer.commit()

total_timeStem = time.time() - start_time
print("Total execution time:", total_timeStem)


Total execution time: 8.544649839401245


In [ ]:
!ls /content/indCorpusPS

_MAIN_1.toc  MAIN_6xwtrgchyph2zc68.seg	MAIN_WRITELOCK


## TF-IDF

In [ ]:
ind = index.open_dir("indCorpusPS")
searcher = ind.searcher(weighting=scoring.TF_IDF())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)
    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)
    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list
total_timeStem_TFIDF = time.time() - start_time
print("Total execution time:", total_timeStem_TFIDF)

Total execution time: 1303.0548467636108


In [ ]:
listAP_PS_TFIDF, listOfAP_PS_Inter, MAP_PS_Inter = MAP(corpus_result_que, results_dict)
l_PS_p5, l_PS_r5, p5_PS, r5_PS = count_P_R(corpus_result_que, results_dict)
listOfAP_PS_NonInter, MAP_PS_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PS_Inter)
print(MAP_PS_NonInter)

0.10729267132498103
0.10192989281227602


In [ ]:
print(p5_PS, r5_PS)

0.01830967247955718 0.8902170756969523


## BM25F

In [ ]:
ind = index.open_dir("indCorpusPS")
searcher = ind.searcher(weighting=scoring.BM25F())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)
    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)
    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list
total_timeStem_BM25F = time.time() - start_time
print("Total execution time:", total_timeStem_BM25F)

Total execution time: 1657.4969823360443


In [ ]:
listAP_PS_BM25F, listOfAP_PS_BM25F_Inter, MAP_PS_BM25F_Inter = MAP(corpus_result_que, results_dict)
l_PS_BM25F_p5, l_PS_BM25F_r5, p5_PS_BM25F, r5_PS_BM25F = count_P_R(corpus_result_que, results_dict)
listOfAP_PS_BM25F_NonInter, MAP_PS_BM25F_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PS_BM25F_Inter)
print(MAP_PS_BM25F_NonInter)

0.17124949206600854
0.16322198388097126


In [ ]:
print(p5_PS_BM25F, r5_PS_BM25F)

0.018331815982534534 0.8918469527581678


## Frequency

In [ ]:
ind = index.open_dir("indCorpusPS")
searcher = ind.searcher(weighting=scoring.Frequency())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in CorpusQueries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)

    num_values = len(corpus_result_que)
    results = searcher.search(query, limit=num_values)

    docid_list = [result['docname'] for result in results]
    results_dict[query_name] = docid_list

total_timeStem_Frequency = time.time() - start_time
print("Total execution time:", total_timeStem_Frequency)

Total execution time: 1074.0482070446014


In [ ]:
listAP_PS_Frequency, listOfAP_PS_Frequency_Inter, MAP_PS_Frequency_Inter = MAP(corpus_result_que, results_dict)
l_PS_Frequency_p5, l_PS_Frequency_r5, p5_PS_Frequency, r5_PS_Frequency = count_P_R(corpus_result_que, results_dict)
listOfAP_PS_Frequency_NonInter, MAP_PS_Frequency_NonInter = MAP(corpus_result_que, results_dict, False)

print(MAP_PS_Frequency_Inter)
print(MAP_PS_Frequency_NonInter)

0.0660052963594649
0.0635549112782788


In [ ]:
print(p5_PS_Frequency, r5_PS_Frequency)

0.018194347927138307 0.8813611289466806


# Thống kê

In [ ]:
import pandas as pd

# Define your data
q1 = ["Stem + Stop + TFIDF", total_timeStemRemoveStop, total_timeStemRemoveStop_TFIDF, MAP_PSRemoveStop_TFIDF_Inter, MAP_PSRemoveStop_TFIDF_NonInter, p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF]
q2 = ["Stem + Stop + BM25F", total_timeStemRemoveStop, total_timeStemRemoveStop_BM25F, MAP_PSRemoveStop_BM25F_Inter, MAP_PSRemoveStop_BM25F_NonInter, p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F]
q3 = ["Stem + Stop + Frequency", total_timeStemRemoveStop, total_timeStemRemoveStop_Frequency, MAP_PSRemoveStop_Frequency_Inter, MAP_PSRemoveStop_Frequency_NonInter, p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency]
q4 = ["Stem + TFIDF", total_timeStem, total_timeStem_TFIDF, MAP_PS_Inter, MAP_PS_NonInter, p5_PS, r5_PS]
q5 = ["Stem + BM25F", total_timeStem, total_timeStem_BM25F, MAP_PS_BM25F_Inter, MAP_PS_BM25F_NonInter, p5_PS_BM25F, r5_PS_BM25F]
q6 = ["Stem + Frequency", total_timeStem, total_timeStem_Frequency, MAP_PS_Frequency_Inter, MAP_PS_Frequency_NonInter, p5_PS_Frequency, r5_PS_Frequency]

col = ["Type", "Index Time", "Query Time", "Inter", "NonInter", "P", "R"]

# Create the DataFrame
df = pd.DataFrame([q1, q2, q3, q4, q5, q6], columns=col)

# Round the numeric columns to 2 decimal places
numeric_cols = ["Index Time", "Query Time", "Inter", "NonInter", "P", "R"]
df[numeric_cols] = df[numeric_cols].round(4)

# Print the rounded DataFrame
df


,Type,Index Time,Query Time,Inter,NonInter,P,R
0,Stem + Stop + TFIDF,15.5184,1087.5344,0.1089,0.1036,0.0184,0.8914
1,Stem + Stop + BM25F,15.5184,1425.4574,0.1749,0.1673,0.0184,0.8928
2,Stem + Stop + Frequency,15.5184,880.9009,0.0670,0.0646,0.0183,0.8817
3,Stem + TFIDF,8.5446,1303.0548,0.1073,0.1019,0.0183,0.8902
4,Stem + BM25F,8.5446,1657.4970,0.1712,0.1632,0.0183,0.8918
5,Stem + Frequency,8.5446,1074.0482,0.0660,0.0636,0.0182,0.8814
